### Storing the query-recipe pairs that were already evaluated as a reference

The ideia is save into a "dictionary" the evaluations of recipes return for a specific query, so that a new model can have the performance on the same set of queries easily compared with the previous ones since most of the answers will be the same.

In [10]:
import pandas as pd

# trying to load the review bank
try:
    review_bank = pd.read_excel('reviews/review_bank.xlsx')
except Exception:
    review_bank = pd.DataFrame()


# loading every other previous evaluation
reviews_paths = {
    "Rodrigo": [
        'reviews/Results_semantic_Rodrigo.xlsx',
        'reviews/Results_Bm25_Rodrigo.xlsx',
        'reviews/Results_Tfidf_Rodrigo.xlsx'
    ],
    "Sanches": [
        'reviews/Results_semantic_sanches.xlsx',
        'reviews/Results_Bm25_sanches.xlsx',
        'reviews/Results_Tfidf_sanches.xlsx'
    ],
    "Vanessa": [
        'reviews/Results_semantic_Vanessa.xlsx',
        'reviews/Results_Bm25_Vanessa.xlsx',
        'reviews/Results_Tfidf_Vanessa.xlsx'
    ],
    "Ramalho": [
        'reviews/Results_Bm25_ramalho.xlsx',
        'reviews/Results_Tfidf_ramalho.xlsx'
    ]
}

for path in reviews_paths['Ramalho']:
    # reading all the reviews
    ramalho_reviews = pd.read_excel(path, sheet_name="Sheet1")
    # renaming the columns
    ramalho_reviews.rename(columns={'Unnamed: 6': 'Nota', 'title': 'Receita'}, inplace=True)
    # subtracting 1 from the ratings because its on 1-6 scale
    ramalho_reviews['Nota'] = ramalho_reviews['Nota'] - 1
    # concatenating the reviews
    review_bank = pd.concat([review_bank, ramalho_reviews[['Query', 'Receita', 'Nota']]])

for path in reviews_paths['Rodrigo']:
    # reading all the reviews
    rod_reviews = pd.read_excel(path, sheet_name="Sheet1")
    # renaming the columns
    rod_reviews.rename(columns={'Unnamed: 12': 'Nota', 'title': 'Receita'}, inplace=True)
    # subtracting 1 from the ratings because its on 1-6 scale
    rod_reviews['Nota'] = rod_reviews['Nota'] - 1
    # concatenating the reviews
    review_bank = pd.concat([review_bank, rod_reviews[['Query', 'Receita', 'Nota']]])

for path in reviews_paths['Sanches']:
    # reading all the reviews
    san_reviews = pd.read_excel(path, sheet_name="Sheet1")
    # renaming the columns
    san_reviews.rename(columns={'nota': 'Nota', 'title': 'Receita'}, inplace=True)
    # subtracting 1 from the ratings because its on 1-6 scale
    san_reviews['Nota'] = san_reviews['Nota'] - 1
    # concatenating the reviews
    review_bank = pd.concat([review_bank, san_reviews[['Query', 'Receita', 'Nota']]])
                
for path in reviews_paths['Vanessa']:
    # reading all the reviews
    van_reviews = pd.read_excel(path, sheet_name="Sheet1")
    # renaming the columns
    van_reviews.rename(columns={'Unnamed: 6': 'Nota', 'title': 'Receita'}, inplace=True)
    # subtracting 1 from the ratings because its on 1-6 scale
    van_reviews['Nota'] = van_reviews['Nota'] - 1
    # concatenating the reviews
    review_bank = pd.concat([review_bank, van_reviews[['Query', 'Receita', 'Nota']]])

# Getting the average rating
review_bank = review_bank.groupby(["Query", "Receita"]).agg({"Nota": "mean"})
review_bank.reset_index(inplace=True)
review_bank["Nota"] = review_bank["Nota"].round().astype(int)
review_bank

,Query,Receita,Nota
0,Brûlée Cream,amazing lavender crme brle,5
1,Brûlée Cream,brled mashed sweet potatoes,3
2,Brûlée Cream,coffee brandy cream brulee,4
3,Brûlée Cream,cream brulee with strawberries,4
4,Brûlée Cream,cream cheese ice cream,0
...,...,...,...
142,what can I make for a romantic dinner,romantic stuffed chicken breasts a deux,5
143,what can I make for a romantic dinner,romantic wild rice salad,5
144,what can I make for a romantic dinner,sensual chicken with champagne,5
145,what can I make for a romantic dinner,valentines day recipe,4


## Exporting the evaluations to a "review bank"

In [11]:
# saving the dataframe to a new excel file
review_bank.to_excel('reviews/review_bank.xlsx', index=False)

review_bank.head()

,Query,Receita,Nota
0,Brûlée Cream,amazing lavender crme brle,5
1,Brûlée Cream,brled mashed sweet potatoes,3
2,Brûlée Cream,coffee brandy cream brulee,4
3,Brûlée Cream,cream brulee with strawberries,4
4,Brûlée Cream,cream cheese ice cream,0


Now for every new evaluation, we can just add the new ratings to this file, and for the new searches we can just filter the recipes that are not in this file to evaluate them.

When the recipes are already on this file, we immediatly have the evaluation.

In [12]:
review_bank = pd.read_excel('reviews/review_bank.xlsx')

def lookup_rating(query, recipe):
    try:
        rating = review_bank[(review_bank['Query'] == query) & (review_bank['Receita'] == recipe)]["Nota"].values[0]
        print(f"The rating of the recipe '{recipe}' for the query '{query}' is {rating}.")	
    except:
        print(f"The recipe '{recipe}' for the query '{query}' was not found in the review bank.")

query = "Brûlée Cream"
recipe = "creme brulee for two"

# checking if the query-recipe pair is in the review bank
lookup_rating(query, recipe)

query = "Lasangna"
recipe = "creme brulee for two"

# checking if the query-recipe pair is in the review bank
lookup_rating(query, recipe)

The rating of the recipe 'creme brulee for two' for the query 'Brûlée Cream' is 5.
The recipe 'creme brulee for two' for the query 'Lasangna' was not found in the review bank.
